In [1]:
import torch
from torch import nn

# Imports for Tensor
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [2]:
#!ls 'gdrive/My Drive/Muller Group Drive/Ear EEG/Drowsiness_Detection/classifier_TBME'
#!ls C:\Users\arya_bastani\Documents\ear_eeg\data\ear_eeg_data
ear_eeg_data_path = '/data/shared/signal-diffusion/eeg_classification_data/ear_eeg_data/ear_eeg_clean'

%ls {ear_eeg_data_path}

In [3]:
# imports
import numpy as np

#import support scripts: pull_data
import support_scripts.read_in_ear_eeg as read_in_ear_eeg
import support_scripts.read_in_labels as read_in_labels
import support_scripts.eeg_filter as eeg_filter

In [4]:
##################
# READ-IN EAR EEG
##################
# NOTE, this takes a long time to run.
# (It could be parallelized to reduce runtime)

# name of spreadsheet with experiment details
# details_spreadsheet = 'gdrive/My Drive/Muller Group Drive/Ear EEG/Drowsiness_Detection/classifier_TBME/classification_scripts/trial_details_spreadsheet_basic.csv'
details_spreadsheet = r'/data/shared/signal-diffusion/eeg_classification_data/ear_eeg_data/trial_details_spreadsheet_good.csv'

# file path to ear eeg data (must be formated r'filepath\\')
#data_filepath = r'C:\Users\Carolyn\OneDrive\Documents\school\berkeley\research\ear_eeg_classification_framework\experimental_recordings\drowsiness_studies\ear_eeg\\'
data_filepath = r'/data/shared/signal-diffusion/eeg_classification_data/ear_eeg_data/ear_eeg_clean//'

# user number or all users('all', 'ryan', 'justin', 'carolyn', 'ashwin', 'connor')
input_users = 'all'

# channels of eeg to read in for each trial (must include 5 and 11 if re-refernecing is enabled in the next block)
data_chs = [1,2,3,4,5,7,8,9,10,11]

# sampling frequency of system (fs=1000 for wandmini)
fs = 1000

# plot eeg data that is read in
plot_raw_data_enable = False

# call read in ear eeg
all_raw_data, filenames, data_lengths, file_users, refs = read_in_ear_eeg.read_in_clean_data(details_spreadsheet, data_filepath, input_users, data_chs, fs, plot_raw_data_enable)
#all_raw_data = np.array(all_raw_data)

In [ ]:
all_raw_data[21]

In [ ]:
#################
# READ-IN LABELS
#################

# Note: label read in will match Ear EEG read in
# (same trials will be read in, and the experiment lengths will be the same)

# file path to labels(must be formated r'filepath\\')
#label_filepath = r'C:\Users\Carolyn\OneDrive\Documents\school\berkeley\research\ear_eeg_classification_framework\experimental_recordings\drowsiness_studies\labels\\'
label_filepath =  r'/data/shared/signal-diffusion/eeg_classification_data/ear_eeg_data/labels//'

# plot the labels that are read in
plot_labels_enable = False

# call read in labels
all_labels = read_in_labels.read_in_labels(filenames, data_lengths, label_filepath, plot_labels_enable)
all_labels = np.array(all_labels)

In [ ]:
print(all_labels[21])

In [ ]:
 = eeg_filter.filter_studies(all_raw_data)


print(len(all_raw_data))
print(all_raw_data[0].shape)

print(len(filtered_data))
print(filtered_data[0].shape)

In [ ]:
# No longer have a need for the original raw data so we delete
del all_raw_data

In [ ]:
import sys
print(sys. getsizeof(filtered_data))

In [ ]:
# Data constants
carolyn_indices = [0,1,2,3,4]
ryan_indices = [5,6,7,8,9]
justin_indices = [10,11,12,13,14]
conor_indices = [15,16,17,18,19]
avi_indices = [20,21]
train_perc, val_perc, test_perc = 0.55, 0.30, .15
train_ind = [2,3,4,8,9,12,13,14,15,17,18,19,21]
val_ind = [1,6,11,16,20,7]
test_ind = [0,5,10]

# Model Constants
window_size = 10 # Seconds
sample_rate = 1000 # Hertz

# Model Params
seq_size = window_size * sample_rate

In [ ]:
# Split up into train, val, and test datasets
train_data, val_data, test_data = [],[],[]
train_labels, val_labels, test_labels = [],[],[]


for i in range(len(filtered_data)):
    if i in train_ind:
        train_data.append(filtered_data[i])
        train_labels.append(all_labels[i])
    elif i in val_ind:
        val_data.append(filtered_data[i])
        val_labels.append(all_labels[i])
    else:
        test_data.append(filtered_data[i])
        test_labels.append(all_labels[i])

In [ ]:
len(train_data[0].T[:,0])

In [ ]:
def formatt_data(data_set, seq_len):
    # Data needs to be input as (samples, channels), for ex: (2,400,000, 10)
    new_dataset = []
    for data in data_set:
        data = data.T
        #print(data.shape)
        new_data = []
        row_len = data.shape[1]
        #print(row_len)
        #print(seq_len)
        num_seqs = int(row_len/seq_len)
        #print("bad")
        for row in data:
            #print(num_seqs)
            new_row = np.split(row, num_seqs)
            new_data.append(new_row)
            #print("bad")
        new_data = np.array(new_data).T
        #print(new_data.shape)
        new_dataset.append(new_data)
        
    return new_dataset

def one_hot_encode(input):
    one_hot_formatted_seq_labels = []
    b = np.zeros((int(input.size), int(input.max() + 1)))
    b[np.arange(input.size), input] = 1
    one_hot_labels = np.array(b)

    return one_hot_labels

def formatt_labels(labels_set, seq_size):
    processed_labels = []
    for i in range(len(labels_set)):
        old_one_hot_labels = one_hot_encode(train_labels[i])
        new_labels = old_one_hot_labels[::seq_size]    
        processed_labels.append(new_labels)
    return processed_labels

In [ ]:
print(np.array(train_data).shape)
print(np.array(train_data).T.shape)

In [ ]:
proc_train = formatt_data(train_data, seq_size)
proc_val = formatt_data(val_data, seq_size)

In [ ]:
print(proc_train[0].shape)

In [ ]:
# format the labels
processed_train_labels = formatt_labels(train_labels, seq_size)
processed_val_labels = formatt_labels(val_labels, seq_size)

In [ ]:
processed_train_labels[0].shape

In [ ]:
proc_train_new = [study[::4] for study in proc_train]
processed_train_labels_new = [study[::4] for study in processed_train_labels]

proc_val_new = [study[::4] for study in proc_val]
processed_val_labels_new = [study[::4] for study in processed_val_labels]

print(proc_train_new[0].shape)

print(processed_train_labels_new[0].shape)

In [ ]:
proc_train_new = [study[:400] for study in proc_train_new]

proc_val_new = [study[:400] for study in proc_val_new]

print(proc_train_new[0].shape)

print(processed_train_labels_new[0].shape)

In [ ]:
print(type(np.double(proc_train_new[0][0][0])[0]))

In [ ]:
from torch.autograd import Variable 

# Turn into tensors

X_train_tensors = [torch.from_numpy(batch) for batch in proc_train_new]
y_train_tensors = [torch.from_numpy(batch) for batch in processed_train_labels_new]

X_val_tensors = [torch.from_numpy(batch) for batch in proc_val_new]
y_val_tensors = [torch.from_numpy(batch) for batch in processed_val_labels_new]

In [ ]:
X_train_tensors[0].double().shape

In [ ]:
# Embedding Layer
# In this implementation we will do a sort of raw embedding for the system
# We will treat each channel as a word, and define the length of each word as the window size we are referencing
# The vectors of each channel will be strung together in order to make a "sentence" which will then have positional 
# encoding added to it, and from there we'll pass it into the 

"""
class AryaEmbed():
    def __init__(vect_len: int):
"""

In [ ]:
"""
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        print("sdapoasd")
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        
        #Args:
        #    x: Tensor, shape [seq_len, batch_size, embedding_dim]
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
        
"""

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, batch_firsty):
        super().__init__()
        
        #self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(1000, hid_dim)  # position embedding
        
        self.encoder_layer = nn.TransformerEncoderLayer(hid_dim, n_heads, pf_dim, dropout, batch_first=batch_firsty)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, n_layers)
        
        self.fc = nn.Linear(input_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src: [src_len, batch_size]
        
        # create position tensor
        #pos = torch.arange(0, src.shape[0]).unsqueeze(1).repeat(1, src.shape[1]).to(src.device)
        
        # embed tokens and positions
        #tok_embedded = self.dropout(self.tok_embedding(src))  # [src_len, batch_size, hid_dim]
        #pos_embedded = self.dropout(self.pos_embedding(pos))  # [src_len, batch_size, hid_dim]
        #embedded = tok_embedded + pos_embedded
        
        # encode sequence
        
        print(type(src))
        
        encoded = self.encoder(src.double())  # [src_len, batch_size, hid_dim]
        
        # get final output and apply linear layer
        final_output = encoded.mean(dim=0)  # [batch_size, hid_dim]
        logits = self.fc(final_output)  # [batch_size, output_dim]
        
        return logits

In [ ]:
# define hyperparameters
INPUT_DIM = 10 #Not needed we're not embedding
OUTPUT_DIM = 2
HID_DIM = INPUT_DIM
N_LAYERS = 4
N_HEADS = 2
PF_DIM = 512
DROPOUT = 0.1
BATCH_FIRST = True #True: (batch, seq, feature). False: (seq, batch, feature)

# create model instance
model = TransformerClassifier(INPUT_DIM, OUTPUT_DIM, HID_DIM, N_LAYERS, N_HEADS, PF_DIM, DROPOUT, BATCH_FIRST)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# define training and evaluation functions
def train(model, X_iterator, y_iterator, optimizer, criterion):
    model.train()
    for batch_num in range(len(X_iterator)):
        src = X_iterator[batch_num].double()
        trg = y_iterator[batch_num].double()
        
        print("src shape: ", src.shape)
        print("src shape: ", trg.shape)

        
        optimizer.zero_grad()
        output = model(src)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
    return loss

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            src, trg = batch
            output = model(src)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [ ]:
# define training loop
N_EPOCHS = 10
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    train_loss = train(model, X_train_tensors, y_train_tensors, optimizer, criterion)
#     valid_loss = evaluate(model, valid_iterator, criterion)
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'model.pt')
#     print(f'Epoch {epoch+1}: train loss={train_loss:.3f}, valid loss={valid_loss:.3f}')

# load best model and evaluate on test set
model.load_state_dict(torch.load('model.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'Test loss={test_loss:.3f}')

In [ ]:
stop

In [ ]:
print(X_train_tensors.shape)
  train_pred = lstm.forward(X_train_tensors) #forward pass
  print("pre soft train_pred: ", train_pred.shape)
  softmaxed_train_pred = train_pred.argmax(dim=2,keepdim=False)
  print("post soft train_pred: ", softmaxed_train_pred.shape)



  optimizer.zero_grad() #caluclate the gradient, manually setting to 0

  #print(type(train_pred[0][0][0]))
  #print(type(y_train_tensors[0][0][0]))

  print(train_pred.shape)
  print(y_train_tensors.shape)


  train_loss = criterion(train_pred, y_train_tensors)
  train_loss.backward() #calculates the loss of the loss function
  optimizer.step() #improve from loss, i.e backprop


  # Run on validation
  test_pred = lstm.forward(X_test_tensors) #forward pass
  #test_loss = criterion(test_pred.type(torch.LongTensor), y_test_tensors.type(torch.LongTensor))
  test_loss = criterion(test_pred, y_test_tensors)

  softmaxed_test_pred = test_pred.argmax(dim=2,keepdim=False)


  print("softmaxed_train_pred; ", softmaxed_train_pred.shape)
  print("y_train: ", y_train.shape)


  train_accuracy = accuracy_score(softmaxed_train_pred, y_train)
  train_accuracies.append(train_accuracy)
  train_losses.append(train_loss.item())

  test_accuracy = accuracy_score(softmaxed_test_pred, y_test)
  test_accuracies.append(test_accuracy)
  test_losses.append(test_loss.item())

  #test_sensitivity, test_specificity = sensitivy_specificity_results(softmaxed_test_pred, y_test, False)
  #test_sensitivities.append(test_sensitivity)
  #test_specificities.append(test_specificity)



In [ ]:
##### TEST SECTION #######

In [ ]:
fake_data = []
for i in range(10):
    start_val = 100000*i
    end_val = start_val + 11000
    row = np.arange(start_val, end_val)
    fake_data.append(row)
fake_data = np.array(fake_data)

print(fake_data.shape)
print(fake_data)



In [ ]:
seq_size = 1000
num_seqs = 10
#new_fake = np.reshape(fake_data, (10,num_seqs, seq_size))


new_fake = []
for row in fake_data:
    new_row = np.split(row,11)
    new_fake.append(new_row)
    
new_fake = np.array(new_fake)  


print(new_fake.shape)
print(new_fake)